<a href="https://colab.research.google.com/github/kfuentestorres/Master-BI-BDA-Project-Work/blob/master/Scraping_WebNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (156 kB/s)
Reading package lists... Done
Reading package lists... Done
Building d

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas
import json
import pprint

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #nasconde browser all'interno del sistema operativo (non vedo che lavora)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


Tgcom

Step 1: ricavo un file con le notizie, la tipologia e la data di pubblicazione; poi seleziono solo gli articoli nel periodo compreso da gennaio a marzo

In [ ]:
wd.get("https://www.tgcom24.mediaset.it/ricerca-mediaset/risultato.shtml?q=coronavirus&fq=prjname%3Atgc&sort=pubdate+asc")

In [ ]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

for i in tqdm(range(1,500)): #mi fermo a 499 iterazioni
  wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  more_buttons = wd.find_element_by_css_selector("#more > a")
  wd.execute_script("arguments[0].click();", more_buttons)
  try:
    WebDriverWait(wd,30).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#more > a")))
  except:
    pass
  time.sleep(1)

list_News = wd.find_elements_by_css_selector("li article")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
Notizie = []
for i in tqdm(list_News):
  try:
    url = i.find_elements_by_css_selector("h3.title a")[0].get_attribute("href")
    tipo = i.find_elements_by_css_selector("h4.type")[0].text
    data = i.find_elements_by_css_selector("div.news__sez > span")[0].text
  except:
    pass
    
  Notizie.append ({'url': url,
                   'tipo': tipo,
                   'data': data})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
import pandas as pd
ds_notizie = pd.DataFrame(Notizie)
ds_notizie.head()

,url,tipo,data
0,https://www.tgcom24.mediaset.it/mondo/cina-ind...,ARTICOLO,09 gen 2020
1,https://www.tgcom24.mediaset.it/mondo/cina-pri...,ARTICOLO,11 gen 2020
2,https://www.tgcom24.mediaset.it/salute/virus-m...,ARTICOLO,19 gen 2020
3,https://www.tgcom24.mediaset.it/mondo/cina-un-...,ARTICOLO,20 gen 2020
4,https://www.tgcom24.mediaset.it/salute/in-16-a...,ARTICOLO,20 gen 2020


In [ ]:
ds_notizie.to_csv('ds_Tgcom24_Notizie.csv')

Step 2: ricavo le informazioni riguardo le notizie selezionate

In [ ]:
import pandas as pd
Notizie = pd.read_csv("ds_Tgcom24_Notizie-2-csv.csv")
Notizie = Notizie['url'].tolist()
#Notizie = Notizie[:10]
print(len(Notizie))

3058


In [ ]:
import time
details_notizie = []
for i in tqdm(Notizie):
  time.sleep(1)
  wd.get(i)
  try:
    id = wd.find_elements_by_css_selector("body.articolo")[0].get_attribute("data-id")
    fonte = "Tgcom24"
    titolo = wd.find_elements_by_css_selector("#cont-article > header > h1")[0].text
    try:
      sottotitolo = wd.find_elements_by_css_selector("#cont-article > header > h2")[0].text
    except:
      sottotitolo = "non presente"
    date = wd.find_elements_by_css_selector("#cont-article > header > time")[0].text
    try:
      body = wd.find_elements_by_css_selector("#cont-article > div.p.p__media__i.p__media__bottom > p")[0].text
      try:
        testo = wd.find_elements_by_css_selector("#cont-article > div:nth-child(10) > p")
        paragrafi = 1
        for j in range(len(testo)):
          body = body + "\n" + testo[j].text
          if len(testo[j].text)>50:
            paragrafi = paragrafi + 1
      except:
        paragrafi = 1
    except:
      paragraph = wd.find_elements_by_css_selector("#cont-article > div.p > p")
      body = ""
      paragrafi = 0
      for k in range(len(paragraph)):
        body = body + "\n" + paragraph[k].text
        if len(paragraph[k].text)>50:
          paragrafi = paragrafi + 1
    try:
      test = wd.find_elements_by_css_selector("#cont-article > div.p.p__media__i.p__media__bottom > div.d__iframe > div")[0]
      video = "presente"
    except:
      video = "non presente"
    try:
      try:
        test = wd.find_elements_by_css_selector("#cont-article > div.p.p__media__i.p__media__bottom > figure > img")[0]
      except:
        test = wd.find_elements_by_css_selector("div.media.media__f.media__f__v2 > div > h3")[0]
      immagine = "presente"
    except:
      immagine = "non presente"
    try:
      test = wd.find_elements_by_css_selector("#cont-article > div.btns.soc")[0]
      share = "presente"
    except:
      test = "non presente"
    url = i
  except:
    pass
    
  details_notizie.append ({'id': id,
                           'fonte': fonte,
                           'titolo': titolo,
                           'suottotitolo': sottotitolo,
                           'body': body,
                           'paragrafi': paragrafi,
                           'date': date,
                           'immagine': immagine,
                           'video': video,
                           'share': share,
                           'url': url})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pandas as pd
ds_details_notizie = pd.DataFrame(details_notizie)
ds_details_notizie.head()

,id,fonte,titolo,suottotitolo,body,paragrafi,date,immagine,video,share,url
0,13224969,Tgcom24,Cina individua una polmonite virale da stesso ...,non presente,La Cina ha individuato in un nuovo tipo di cor...,1,09 GENNAIO 2020 06:57,presente,non presente,presente,https://www.tgcom24.mediaset.it/mondo/cina-ind...
1,13306839,Tgcom24,"Cina, primo morto per polmonite da virus della...",Tra il 2002 e il 2003 la Sars uccise oltre 700...,La Cina ha annunciato il primo decesso legato ...,7,11 GENNAIO 2020 19:30,non presente,presente,presente,https://www.tgcom24.mediaset.it/mondo/cina-pri...
2,13634502,Tgcom24,Virus misterioso in Cina: i casi sarebbero 1.7...,Si tratta di un nuovo ceppo di coronavirus che...,"I contagi del ""virus misterioso"" che ha causat...",4,19 GENNAIO 2020 07:06,presente,non presente,presente,https://www.tgcom24.mediaset.it/salute/virus-m...
3,13696765,Tgcom24,"Virus Cina, confermato il passaggio da uomo a ...",Cresce lʼallarme per la diffusione della malat...,"Il nuovo coronavirus, della stessa famiglia de...",6,21 GENNAIO 2020 01:38,non presente,presente,presente,https://www.tgcom24.mediaset.it/mondo/cina-un-...
4,13726854,Tgcom24,In 16 anni sono 5 i virus diventati capaci di ...,"Secondo gli esperti, ciò costituisce ""un forte...",In sedici anni sono cinque i virus che hanno f...,7,20 GENNAIO 2020 18:00,presente,non presente,presente,https://www.tgcom24.mediaset.it/salute/in-16-a...


In [ ]:
ds_details_notizie.to_csv('ds_Tgcom24.csv')

La Stampa

In [ ]:
import time
Notizie = []
for i in tqdm(range(560,726)):
  time.sleep(1)
  wd.get(f"https://www.lastampa.it/ricerca?query=covid&page={i}")
  list_News = wd.find_elements_by_css_selector("div h2.entry__title")
  for i in list_News:
    try:
      url = i.find_elements_by_css_selector("a")[0].get_attribute("href")
      if len([x for x in url.split('/') if x == 'topnews'])==0 and len([x for x in url.split('/') if x == 'opinioni'])==0:
        Notizie.append(url)
    except:
      pass
print(len(Notizie))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



166


In [ ]:
import time
import pandas as pd
details_notizie = []
for i in tqdm(Notizie):
  time.sleep(1)
  wd.get(i)
  try:
    fonte = "La Stampa"
    id = i    
    titolo = wd.find_elements_by_css_selector("article > header > h2.entry__title")[0].text
    try:
      sottotitolo = wd.find_elements_by_css_selector("article > header > p.entry__subtitle")[0].text
    except:
      sottotitolo = "non presente"
    paragraph = wd.find_elements_by_css_selector("div.entry__content > p")
    body = ""
    paragrafi = 0
    for j in range(len(paragraph)):
      if len(paragraph[j].text)>70:
        body = body + "\n" + paragraph[j].text
        paragrafi = paragrafi + 1
    date = wd.find_elements_by_css_selector("span.entry__date")[0].text
    try:
      ora = wd.find_elements_by_css_selector("span.entry__date.entry__date--update > i")[0].text
    except:
      ora = "non presente"
    try:
      autore = wd.find_elements_by_css_selector("span.entry__author")[0].text
    except:
      autore = "non presente"
    try:
      test = wd.find_elements_by_css_selector("article > header > figure > picture > img")[0]
      immagine = "presente"
    except:
      immagine = "non presente"
    try:
      test = wd.find_elements_by_css_selector("div.responsive-video__title")[0].text
      video = "presente"
    except:
      video = "non presente"
    try:
      test = wd.find_elements_by_css_selector("span.share-text")[0]
      share = "presente"
    except:
      share = "non presente"
    url = i
  except:
    pass
  details_notizie.append ({'id': id,
                           'fonte': fonte,
                           'titolo': titolo,
                           'sottotitolo': sottotitolo,
                           'body': body,
                           'paragrafi': paragrafi,
                           'date': date,
                           'ora': ora,
                           'autore': autore,
                           'immagine': immagine,
                           'video': video,
                           'share': share,
                           'url': url})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
import pandas as pd
ds_details_notizie = pd.DataFrame(details_notizie)
ds_details_notizie.head()

,id,fonte,titolo,sottotitolo,body,paragrafi,date,ora,autore,immagine,video,share,url
0,https://www.lastampa.it/torino/2020/04/01/news...,La Stampa,"Coronavirus, cinquantamila euro donati dai val...",,\nSUSA (TORINO) Ammonta ad oltre 26 mila euro ...,3,PUBBLICATO IL\n01 Aprile 2020,11:04,FRANCESCO FALCONE,presente,non presente,presente,https://www.lastampa.it/torino/2020/04/01/news...
1,https://www.lastampa.it/torino/2020/04/01/news...,La Stampa,In ospedale per il coronavirus gli scoprono un...,Tre ospedali torinesi coinvolti: la massa tumo...,"\nTORINO. Una storia incredibile, che di racco...",3,PUBBLICATO IL\n01 Aprile 2020,13:04,ALESSANDRO MONDO,presente,presente,presente,https://www.lastampa.it/torino/2020/04/01/news...
2,https://www.lastampa.it/cuneo/2020/04/01/news/...,La Stampa,"Fossano, l’opposizione: “Destinare un hotel a ...",,\nDopo aver chiesto l’istituzione di un tavolo...,4,PUBBLICATO IL\n01 Aprile 2020,11:04,ALBERTO PRIERI,presente,non presente,presente,https://www.lastampa.it/cuneo/2020/04/01/news/...
3,https://www.lastampa.it/imperia-sanremo/2020/0...,La Stampa,“Ogni chiamata è sospetta. Non dobbiamo mollar...,Asl Imperiese: a marzo 700 interventi per “evi...,\n«Al punto in cui siamo arrivati ogni chiamat...,2,PUBBLICATO IL\n01 Aprile 2020,non presente,GIULIO GAVINO,presente,non presente,presente,https://www.lastampa.it/imperia-sanremo/2020/0...
4,https://www.lastampa.it/cronaca/2020/04/01/new...,La Stampa,"Coronavirus, nel Lazio contagi in frenata “ma ...",Oggi 169 positivi rispetto ai 181 di ieri. Il ...,\nROMA. «Oggi registriamo un dato di 169 casi ...,6,PUBBLICATO IL\n01 Aprile 2020,18:04,LUISA MOSELLO,presente,presente,presente,https://www.lastampa.it/cronaca/2020/04/01/new...


In [ ]:
ds_details_notizie.to_csv('ds_LaStampa.csv')

Libero

In [ ]:
import time
Notizie = []
for i in tqdm(range(162,302)):
  time.sleep(1)
  wd.get(f"https://www.liberoquotidiano.it/tag/coronavirus/page/{i}/")
  list_News = wd.find_elements_by_css_selector("article.box header")
  for i in list_News:
    try:
      url = i.find_elements_by_css_selector("a")[0].get_attribute("href")
      if len([x for x in url.split('-') if x == 'sondaggio'])==0 and len([x for x in url.split('/') if x == 'sondaggi'])==0:
        Notizie.append(url)
    except:
      pass
print(len(Notizie))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



2058


In [ ]:
import time
details_notizie = []
for i in tqdm(Notizie):
  time.sleep(1)
  wd.get(i)
  try:
    fonte = "Libero"
    id = i
    paragrafi = 0
    try:
      titolo = wd.find_elements_by_css_selector("body > main > h1")[0].text
      paragraph = wd.find_elements_by_css_selector("body > main > div > section > p")
      body = ""
      for j in range(len(paragraph)):
        if len(paragraph[j].text)>50:
          body = body + "\n" + paragraph[j].text
          paragrafi = paragrafi + 1
      video = "non presente"
      try:
        autore = wd.find_elements_by_css_selector("div.article-data a.author")[0].text
      except:
        autore = "non presente"
    except:
      video = "presente"
      titolo = wd.find_elements_by_css_selector("body > main > div.video-description > h1")[0].text
      body = wd.find_elements_by_css_selector("body > main > div.video-description > p")[0].text
      paragrafi = 1
      video = "presente"
      autore = "non presente"
    date = wd.find_elements_by_css_selector("div time")[0].text
    try:
      test = wd.find_elements_by_css_selector("body main section > figure")[0]
      immagine = "presente"
    except:
      immagine = "non presente"
    try:
      test = wd.find_elements_by_css_selector("ul.custom_images")[0]
      share = "presente"
    except:
      share = "non presente"
    url = i
  except:
    pass
  details_notizie.append ({'id': id,
                           'fonte': fonte,
                           'titolo': titolo,
                           'body': body,
                           'paragrafi': paragrafi,
                           'date': date,
                           'autore': autore,
                           'immagine': immagine,
                           'video': video,
                           'share': share,
                           'url': url})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pandas as pd
ds_details_notizie = pd.DataFrame(details_notizie)
ds_details_notizie.head()

,id,fonte,titolo,body,paragrafi,date,autore,immagine,video,share,url
0,https://www.liberoquotidiano.it/news/sport/216...,Libero,"Coronavirus, idea tedesca: chiudere tutte le s...","\nChristian Seifert, amministratore delegato d...",2,01 aprile 2020,,presente,non presente,presente,https://www.liberoquotidiano.it/news/sport/216...
1,https://www.liberoquotidiano.it/news/spettacol...,Libero,"Iene, il programma di Italia 1 non torna ancor...","\nIl ritorno de Le Iene, dopo il periodo di qu...",2,01 aprile 2020,,presente,non presente,presente,https://www.liberoquotidiano.it/news/spettacol...
2,https://www.liberoquotidiano.it/news/personagg...,Libero,Domenico Pallaria si dimette da commissario in...,"\nDomenico Pallaria, commissario della Protezi...",2,01 aprile 2020,,presente,non presente,presente,https://www.liberoquotidiano.it/news/personagg...
3,https://www.liberoquotidiano.it/news/personagg...,Libero,"Franco Bechis: coronavirus, clamorosa cantonat...",\nPer una settimana ai malati di coronavirus è...,2,01 aprile 2020,,presente,non presente,presente,https://www.liberoquotidiano.it/news/personagg...
4,https://www.liberoquotidiano.it/news/commenti-...,Libero,Vittorio Feltri e l'ospedale alla Fiera di Mil...,\nLa Lombardia e il Veneto hanno subito attacc...,2,01 aprile 2020,Vittorio Feltri,presente,non presente,presente,https://www.liberoquotidiano.it/news/commenti-...


In [ ]:
ds_details_notizie.to_csv('ds_Libero.csv')

Il Post

In [ ]:
import time
Notizie = {}
for i in tqdm(range(45,70)):
  time.sleep(1)
  wd.get(f"https://www.ilpost.it/tag/coronavirus/page/{i}/")
  list_News = wd.find_elements_by_css_selector("div article")
  for i in list_News:
    try:
      url = i.find_elements_by_css_selector("h2.entry-title a")[0].get_attribute("href")
      id = i.get_attribute("id")
      Notizie.update({url:id})
    except:
      pass
print(len(Notizie))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



634


In [ ]:
import time
details_notizie = []
for i in tqdm(Notizie):
  time.sleep(1)
  wd.get(i)
  try:
    fonte = "Il Post"
    id = Notizie.get(i)
    titolo = wd.find_elements_by_css_selector("h1.entry-title")[0].text
    try:
      sottotitolo = wd.find_elements_by_css_selector("div.sottit")[0].text
    except:
      sottotitolo = "non presente"
    paragraph = wd.find_elements_by_css_selector("#singleBody > p")
    body = ""
    paragrafi = 0
    for j in range(len(paragraph)):
      if len(paragraph[j].text)>50:
        body = body + "\n" + paragraph[j].text
        paragrafi = paragrafi + 1
    date = wd.find_elements_by_css_selector("span.intera")[0].text
    try:
      immagine = wd.find_elements_by_css_selector("#photosize")[0]
      immagine = "presente"
    except:
      immagine = "non presente"
    try:
      grafico = wd.find_elements_by_css_selector("div.dw-chart")[0]
      grafico = "presente"
    except:
      grafico = "non presente"
    try:
      share = wd.find_elements_by_css_selector("div.social")[0]
      share = "presente"
    except:
      share = "non presente"
    url = i
  except:
    pass
  details_notizie.append ({'id': id,
                           'fonte': fonte,
                           'titolo': titolo,
                           'sottotitolo': sottotitolo,
                           'body': body,
                           'paragrafi': paragrafi,
                           'date': date,
                           'immagine': immagine,
                           'grafico': grafico,
                           'share': share,
                           'url': url})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pandas as pd
ds_details_notizie = pd.DataFrame(details_notizie)
ds_details_notizie.head()

,id,fonte,titolo,sottotitolo,body,paragrafi,date,immagine,grafico,share,url
0,post-2267196,Il Post,Circolare ministeriale sulla passeggiata,non presente,\nFase uno\nNella giornata del 31 marzo il Min...,14,VENERDÌ 3 APRILE 2020,non presente,non presente,presente,https://www.ilpost.it/?blog_post=circolare-min...
1,post-2267138,Il Post,Cos’è questa storia del “coronavirus nell’aria”,È stata segnalata da diversi ricercatori e rip...,\nOggi diversi giornali hanno ripreso – con qu...,19,VENERDÌ 3 APRILE 2020,presente,non presente,presente,https://www.ilpost.it/2020/04/03/coronavirus-a...
2,post-2267024,Il Post,"La proposta di una cassa integrazione europea,...",È la più innovativa fra quelle avanzate dalla ...,"\nMercoledì la Commissione Europea, l’organo e...",11,VENERDÌ 3 APRILE 2020,presente,non presente,presente,https://www.ilpost.it/2020/04/03/cassa-integra...
3,post-2267035,Il Post,La dura risposta del ministero della Difesa ru...,non presente,\nIl portavoce del ministero della Difesa russ...,8,VENERDÌ 3 APRILE 2020,non presente,non presente,presente,https://www.ilpost.it/2020/04/03/ministero-dif...
4,post-2267015,Il Post,«Qui il virus non c’è»,"Lo dice il presidente della Bielorussia, Alexa...","\nIl presidente della Bielorussia, Alexander L...",6,VENERDÌ 3 APRILE 2020,presente,non presente,presente,https://www.ilpost.it/2020/04/03/bielorussia-c...


In [ ]:
ds_details_notizie.to_csv('ds_IlPost.csv')

Il Fatto Quotidiano

In [ ]:
import time
Notizie = []
for i in tqdm(range(424,650)):
  time.sleep(1)
  wd.get(f"https://www.ilfattoquotidiano.it/tag/coronavirus/page/{i}/")
  list_News = wd.find_elements_by_css_selector("h3.p-item")
  for i in list_News:
    try:
      url = i.find_elements_by_css_selector("a")[0].get_attribute("href")
      Notizie.append(url)
    except:
      pass
print(len(Notizie))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



2257


In [ ]:
import time
details_notizie = []
for i in tqdm(Notizie):
  time.sleep(1)
  wd.get(i)
  try:
    fonte = "Il Fatto Quotidiano"
    id = wd.find_elements_by_css_selector("section.article-body")[0].get_attribute("id")
    titolo = wd.find_elements_by_css_selector("h1.title-article")[0].text
    try:
      sottotitolo = wd.find_elements_by_css_selector("section.catenaccio")[0].text
    except:
      sottotitolo = "non presente"
    paragraph = wd.find_elements_by_css_selector("section.article-body p")
    body = ""
    paragrafi = 0
    for j in range(len(paragraph)-2):
      if len(paragraph[j].text)>50:
        body = body + "\n" + paragraph[j].text
        paragrafi = paragrafi + 1
    date = wd.find_elements_by_css_selector("span.date")[0].text
    if wd.find_elements_by_css_selector("div.wrapper-info-article > cite > a")[0].text.replace(" ","").lower() == "dif.q.":
      autore = "non presente"
    else:
      autore = wd.find_elements_by_css_selector("div.wrapper-info-article > cite > a")[0].text
    try:
      immagine = wd.find_elements_by_css_selector("picture.main-image-article")[0]
      immagine = "presente"
    except:
      immagine = "non presente"
    try:
      video = wd.find_elements_by_css_selector("div.jw-media > video")[0]
      video = "presente"
    except:
      video = "non presente"
    try:
      share = wd.find_elements_by_css_selector("div.social-article-wrapper li.share-count-item")[0]
      share = "presente"
    except:
      share = "non presente"
    url = i
  except:
    pass
  details_notizie.append ({'id': id,
                           'fonte': fonte,
                           'titolo': titolo,
                           'sottotitolo': sottotitolo,
                           'body': body,
                           'paragrafi': paragrafi,
                           'date': date,
                           'autore': autore,
                           'immagine': immagine,
                           'video': video,
                           'share': share,
                           'url': url})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pandas as pd
ds_details_notizie = pd.DataFrame(details_notizie)
ds_details_notizie.head()

,id,fonte,titolo,sottotitolo,body,paragrafi,date,autore,immagine,video,share,url
0,article-body-5756408,Il Fatto Quotidiano,Barbara Palombelli e la circolare del Viminale...,La conduttrice di Stasera Italia su Rete4 ha f...,\nBarbara Palombelli legge la circolare dirama...,1,| 1 APRILE 2020,non presente,presente,non presente,presente,https://www.ilfattoquotidiano.it/2020/04/01/ba...
1,article-body-5756406,Il Fatto Quotidiano,"Coronavirus, Acea: “lo stop alla produzione eu...",L'associazione dei costruttori europei fa il p...,\nNel giorno in cui arrivano i disastrosi dati...,4,| 1 APRILE 2020,non presente,presente,non presente,presente,https://www.ilfattoquotidiano.it/2020/04/01/co...
2,article-body-5756369,Il Fatto Quotidiano,"Coronavirus, Speranza: “Strada da percorrere è...",non presente,"\n“Attenzione, non dobbiamo confondere i primi...",1,| 1 APRILE 2020,non presente,non presente,presente,presente,https://www.ilfattoquotidiano.it/2020/04/01/co...
3,article-body-5756384,Il Fatto Quotidiano,"Coronavirus, come far ripartire il calcio? Tar...",Il campione del mondo '82 racconta al Fatto.it...,\nIl soprannome glielo ha dato il suo compagno...,3,| 1 APRILE 2020,di Alberto Facchinetti,presente,non presente,presente,https://www.ilfattoquotidiano.it/2020/04/01/co...
4,article-body-5756016,Il Fatto Quotidiano,"Sono le Venti (Nove), dal nuovo ospedale in Fi...",non presente,"\nRivedi l’ultima puntata di Sono le Venti, la...",2,| 1 APRILE 2020,non presente,non presente,non presente,presente,https://www.ilfattoquotidiano.it/2020/04/01/so...


In [ ]:
ds_details_notizie.to_csv('ds_IlFattoQuotidiano.csv')

La Repubblica

In [ ]:
import time
Notizie = []
for i in tqdm(range(1,327)):
  time.sleep(1)
  wd.get(f"https://ricerca.repubblica.it/ricerca/repubblica-it?author=&sortby=adate&query=coronavirus&fromdate=2020-01-01&fromdate=2020-01-01&todate=2020-03-31&todate=2020-03-31&filter_type=articolo&mode=any&page={i}")
  list_News = wd.find_elements_by_css_selector("div article")
  for i in list_News:
    try:
      extra = i.find_elements_by_css_selector("span.extra")[0].text
    except:
      try:
        url = i.find_elements_by_css_selector("h1 a")[0].get_attribute("href")
        Notizie.append(url)
      except:
        pass
print(len(Notizie))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



2500


In [ ]:
import time
detail_notizie = []
for i in tqdm(Notizie):
  time.sleep(1)
  wd.get(i)
  if len([x for x in i.split('/') if x == 'dossier'])!=0:
    try:
      fonte = "La Repubblica"
      id = i
      try:
        titolo = wd.find_elements_by_css_selector("div > header > h1")[0].text
      except:
        titolo = "non presente"
      try:
        sottotitolo = wd.find_elements_by_css_selector("article div.detail_summary")[0].text
      except:
        sottotitolo = "non presente"
      testo = wd.find_elements_by_css_selector("article  div.detail_body")[0].text
      paragrafi = 0
      body = ""
      for j in testo.split("\n"):
        if len(j)>70:
          body = body + "\n" + j
        if len(j)>100:
          paragrafi = paragrafi + 1
      date = wd.find_elements_by_css_selector("div time")[0].text
      try:
        autore = wd.find_elements_by_css_selector("article > div  header > em")[0].text
      except:
        autore = "non presente"
      try:
        try:
          test = wd.find_elements_by_css_selector("article div > figure > picture > img")[0]
        except:
          test = wd.find_elements_by_css_selector("div.body-text > figure > img")[0]
        immagine = "presente"
      except:
        immagine = "non presente"
      try:
        test = wd.find_elements_by_css_selector("section.inline-video > h1")[0]
        video = "presente"
      except:
        video = "non presente"
      try:
        test = wd.find_elements_by_css_selector("span.share-text")[0].text
        share = "presente"
      except:
        share = "non presente"
      url = i
    except:
      pass
  else:
    try:
      fonte = "La Repubblica"
      id = i
      try:
        titolo = wd.find_elements_by_css_selector("article > header > h1")[0].text
      except:
        titolo = "non presente"
      try:
        sottotitolo = wd.find_elements_by_css_selector("article > header > p")[0].text
      except:
        sottotitolo = "non presente"
      testo = wd.find_elements_by_css_selector("div.body-text")[0].text
      paragrafi = 0
      body = ""
      for j in testo.split("\n"):
        if len(j)>70:
          body = body + "\n" + j
        if len(j)>100:
          paragrafi = paragrafi + 1
      date = wd.find_elements_by_css_selector("div time")[0].text
      try:
        autore = wd.find_elements_by_css_selector("article > header > em > span")[0].text
      except:
        autore = "non presente"
      try:
        try:
          test = wd.find_elements_by_css_selector("article > header > figure > img")[0]
        except:
          test = wd.find_elements_by_css_selector("div.body-text > figure > img")[0]
        immagine = "presente"
      except:
        immagine = "non presente"
      try:
        test = wd.find_elements_by_css_selector("section.inline-video > h1")[0]
        video = "presente"
      except:
        video = "non presente"
      try:
        test = wd.find_elements_by_css_selector("span.share-text")[0].text
        share = "presente"
      except:
        share = "non presente"
      url = i
    except:
      pass
  detail_notizie.append ({'id': id,
                          'fonte': fonte,
                          'titolo': titolo,
                          'sottotitolo':sottotitolo,
                          'body': body,
                          'paragrafi': paragrafi,
                          'date': date,
                          'autore': autore,
                          'immagine': immagine,
                          'video': video,
                          'share': share,
                          'url': url})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pandas as pd
ds_detail_notizie = pd.DataFrame(detail_notizie)
ds_detail_notizie.head()

,id,fonte,titolo,sottotitolo,body,paragrafi,date,autore,immagine,video,share,url
0,https://www.repubblica.it/esteri/2020/01/11/ne...,La Repubblica,"Cina, morto primo paziente affetto da polmonit...",Dietro alla polmonite virale diffusasi in Cina...,\nUn paziente ricoverato per avere contratto l...,5,11 gennaio 2020,non presente,presente,non presente,presente,https://www.repubblica.it/esteri/2020/01/11/ne...
1,https://www.repubblica.it/salute/medicina-e-ri...,La Repubblica,Cina: completata mappa genetica del nuovo viru...,Dal mese di dicembre 2019 ha provocato 41 casi...,\nE' STATA completata la mappa genetica del vi...,7,12 gennaio 2020,non presente,presente,non presente,presente,https://www.repubblica.it/salute/medicina-e-ri...
2,https://www.repubblica.it/salute/medicina-e-ri...,La Repubblica,"Nuovo virus in Cina, primo caso isolato in Tha...",In un viaggiatore proveniente dalla città cine...,\nE' stato identificato in Thailandia il primo...,9,13 gennaio 2020,non presente,presente,non presente,presente,https://www.repubblica.it/salute/medicina-e-ri...
3,https://www.repubblica.it/esteri/2020/01/17/ne...,La Repubblica,"Cina, seconda morte per la polmonite misterios...",Vittima un uomo di 69 anni. La causa sarebbe i...,\nUna seconda persona è morta per la misterios...,7,17 gennaio 2020,non presente,presente,non presente,presente,https://www.repubblica.it/esteri/2020/01/17/ne...
4,https://www.repubblica.it/salute/medicina-e-ri...,La Repubblica,"Cina, possibili 1700 casi di infezione legati ...",La stima degli esperti britannici sul coronavi...,\nPotrebbero essere 1.723 i casi di infezione ...,13,18 gennaio 2020,non presente,presente,non presente,presente,https://www.repubblica.it/salute/medicina-e-ri...


In [ ]:
ds_detail_notizie.to_csv('ds_LaRepubblica.csv')